In [1]:
import copy
from collections import Counter

In [2]:
def translate(vaddr_str):
    vaddr = int(vaddr_str, base=16)
    if 0x80000000 <= vaddr <= 0x9FFFFFFF:
        addr = vaddr - 0x80000000
    elif 0xA0000000 <= vaddr <= 0xBFFFFFFF:
        addr = vaddr - 0xA0000000
    else:
        raise ValueError(vaddr_str)
    return '{:08x}'.format(addr)

In [3]:
# 读取每一行一开始的 PC 值，返回一个列表
def read_instructions(fp):
    return [
        translate(line.split(':')[0])
        for line in fp
        if ':' in line
    ]

In [4]:
# 指定测试点
target_name = 'coremark'
target = f'{target_name}/{target_name}'

lw_file_fp = open(f'{target}.lw', 'r')
sw_file_fp = open(f'{target}.sw', 'r')
trace_fp = open(f'{target}.txt', 'r')

traces = list(map(lambda x: x.split(','), trace_fp.read().split('\n')))

# 使用 .sw 和 .lw 文件来辅助判断某条指令是不是 lw/sw
# 因为 trace 中只会记录指令的 PC，而不会记录指令的编码，
# 所以我们只能利用反汇编文件，用 PC 找到对应的指令是谁，
# 从而确定是不是 lw/sw。
sw_list = read_instructions(sw_file_fp)
lw_list = read_instructions(lw_file_fp)
sw_set = set(sw_list)
lw_set = set(lw_list)

In [5]:
x1faf_read_count = 0
x1faf_write_count = 0
x1fc_read_count = 0
x1fc_write_count = 0
instr_count = 0
sw_count = 0
lw_count = 0
pfn_count = Counter()
tags = []
for i in range(2**6):
    tags.append(Counter())

last_vaddr = None
for rec in traces:
    if len(rec) == 1:
        continue
    elif len(rec) == 2:
        addr, channel = rec

        if not addr.startswith('1faf'):
            index = (int(addr, base=16) >> 6) & 0x3f
            tag = addr[:5]
            tags[index].update({tag: 1})
            pfn_count.update({tag: 1})

        if addr.startswith('1faf'):
            if channel == 'aw':
                x1faf_write_count += 1
            else:
                x1faf_read_count += 1
        elif addr.startswith('1fc'):
            if channel == 'aw':
                x1fc_write_count += 1
            else:
                x1fc_read_count += 1
        else:
            raise RuntimeError(f'AXI: {addr}')
    else:
        vaddr, write_en, reg_num, data = rec
        if vaddr == last_vaddr:
            continue
        last_vaddr = vaddr
        instr_count += 1

        paddr = translate(vaddr)
        pfn_count.update({paddr[:5]: 1})

        if paddr in sw_set:
            sw_count += 1
        elif paddr in lw_set:
            lw_count += 1

# 打印所有 *_count 变量
local_map = copy.copy(locals())
for key, value in local_map.items():
    if key.endswith('count'):
        print(f'{key}: {value}')
del local_map

print(f'#pfn: {len(pfn_count)}')

x1faf_read_count: 9
x1faf_write_count: 920
x1fc_read_count: 478975
x1fc_write_count: 16460
instr_count: 412268
sw_count: 14187
lw_count: 39027
pfn_count: Counter({'1fc09': 225666, '1fc07': 217798, '1fc05': 122702, '1fc04': 89720, '1fc06': 79070, '1fc1a': 76999, '1fc08': 53418, '1fc0a': 37959, '1fc01': 3572, '1fc00': 799})
#pfn: 10


In [6]:
print(f'max associativity: {max(len(x) for x in tags)}')
for i in range(2**6):
    print(f'{i}: [{len(tags[i])}] {tags[i]}')

max associativity: 9
0: [7] Counter({'1fc06': 271, '1fc08': 64, '1fc0a': 35, '1fc09': 31, '1fc00': 17, '1fc07': 12, '1fc01': 8})
1: [7] Counter({'1fc0a': 5449, '1fc09': 884, '1fc06': 258, '1fc08': 64, '1fc07': 16, '1fc01': 16, '1fc00': 12})
2: [7] Counter({'1fc0a': 4603, '1fc05': 1524, '1fc07': 984, '1fc09': 756, '1fc08': 384, '1fc06': 240, '1fc01': 6})
3: [6] Counter({'1fc08': 4208, '1fc0a': 3158, '1fc05': 1296, '1fc06': 710, '1fc07': 601, '1fc09': 58})
4: [7] Counter({'1fc06': 1659, '1fc08': 1332, '1fc05': 1080, '1fc09': 512, '1fc0a': 128, '1fc07': 16, '1fc01': 5})
5: [8] Counter({'1fc08': 4436, '1fc06': 1087, '1fc05': 1012, '1fc09': 941, '1fc0a': 256, '1fc04': 32, '1fc01': 16, '1fc07': 8})
6: [8] Counter({'1fc08': 1536, '1fc06': 1089, '1fc05': 648, '1fc09': 643, '1fc0a': 620, '1fc04': 64, '1fc07': 15, '1fc01': 4})
7: [7] Counter({'1fc05': 1528, '1fc06': 1012, '1fc0a': 748, '1fc08': 460, '1fc04': 416, '1fc09': 24, '1fc07': 5})
8: [7] Counter({'1fc05': 4860, '1fc08': 1648, '1fc06': 11

In [7]:
axi_count = x1faf_read_count + x1faf_write_count + x1fc_read_count + x1fc_write_count
print(f'axi_count = {axi_count}')

axi_count = 496364


In [8]:
cpu_count = instr_count + sw_count + lw_count
print(f'cpu_count = {cpu_count}')

cpu_count = 465482


In [9]:
diff = axi_count - cpu_count
print(f'diff = {diff}')

diff = 30882
